In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_user = ""
db_password = ""
db_host = ""  
db_port = "" 
db_name = ""
connection = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection)

In [3]:
from datetime import datetime
citta = '08|037|006%' #Bologna
intervallo_inizio = datetime(2019, 8, 1,  0,  0,  0)   # 2019-08-01 00:00:00
intervallo_fine   = datetime(2019, 9, 30, 0, 0,  0)   # 2019-09-30 00:00:00
limite = 10 # quanti flussi visualizzare

In [4]:
query = text("""
    SELECT
        CASE WHEN layerid < toid THEN layerid ELSE toid END AS ace1,
        CASE WHEN layerid < toid THEN toid ELSE layerid END AS ace2,
        SUM(datavalue) AS total_movements
    FROM movements
    WHERE layerid LIKE :citta
      AND toid LIKE :citta
      AND datefrom >= :inizio
      AND dateto   <= :fine
    GROUP BY ace1, ace2
    ORDER BY total_movements DESC
    LIMIT :limite
""")

try:
    with engine.connect() as conn:
        df = pd.read_sql(query, conn, params={
            "citta":   f"%{citta}%",
            "inizio":  intervallo_inizio,
            "fine":    intervallo_fine,
            "limite": limite
        })
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")

                 ace1                ace2  total_movements
0  08|037|006|006|006  08|037|006|007|007           430844
1  08|037|006|003|003  08|037|006|007|007           372190
2  08|037|006|003|003  08|037|006|003|031           329693
3  08|037|006|005|051  08|037|006|007|071           321879
4  08|037|006|007|007  08|037|006|007|072           317033
5  08|037|006|006|006  08|037|006|006|062           304500
6  08|037|006|007|071  08|037|006|007|072           266982
7  08|037|006|003|003  08|037|006|006|006           253291
8  08|037|006|003|031  08|037|006|008|008           246912
9  08|037|006|002|002  08|037|006|002|022           238789


In [5]:
# 1. Scarico solo le geometrie
query_geom =text("""SELECT layerid, geojson FROM geom_data WHERE layerid LIKE '""" + citta + """' """)
df_geom = pd.read_sql(query_geom, engine)

# 2. Rinomino per i join successivi
df_geom_from = df_geom.rename(columns={'layerid': 'ace1', 'geojson': 'geo1'})
df_geom_to = df_geom.rename(columns={'layerid': 'ace2', 'geojson': 'geo2'})

# 3. Faccio le due join in pandas
df = df.merge(df_geom_from, on='ace1', how='left')
df = df.merge(df_geom_to, on='ace2', how='left')
df

,ace1,ace2,total_movements,geo1,geo2
0,08|037|006|006|006,08|037|006|007|007,430844,"{'id': 'a40f262e-035f-432e-a287-8046a2ba2851',...","{'id': '81eb35d9-47cd-4394-a40d-8ab998b2b704',..."
1,08|037|006|003|003,08|037|006|007|007,372190,"{'id': '285935bc-b278-4ad4-bae8-ba42b7fb03da',...","{'id': '81eb35d9-47cd-4394-a40d-8ab998b2b704',..."
2,08|037|006|003|003,08|037|006|003|031,329693,"{'id': '285935bc-b278-4ad4-bae8-ba42b7fb03da',...","{'id': '846465de-75b4-47b5-84fc-f4bfc1d59d20',..."
3,08|037|006|005|051,08|037|006|007|071,321879,"{'id': 'ed80820a-ed77-4506-8fd2-7558b291f2aa',...","{'id': '6db59a42-ee90-41d0-9d1f-334bb1747742',..."
4,08|037|006|007|007,08|037|006|007|072,317033,"{'id': '81eb35d9-47cd-4394-a40d-8ab998b2b704',...","{'id': '902337d0-41d9-4641-bbaf-789553cf6593',..."
5,08|037|006|006|006,08|037|006|006|062,304500,"{'id': 'a40f262e-035f-432e-a287-8046a2ba2851',...","{'id': 'eae43f3a-5b56-4688-9c3c-3b24f6d5c22b',..."
6,08|037|006|007|071,08|037|006|007|072,266982,"{'id': '6db59a42-ee90-41d0-9d1f-334bb1747742',...","{'id': '902337d0-41d9-4641-bbaf-789553cf6593',..."
7,08|037|006|003|003,08|037|006|006|006,253291,"{'id': '285935bc-b278-4ad4-bae8-ba42b7fb03da',...","{'id': 'a40f262e-035f-432e-a287-8046a2ba2851',..."
8,08|037|006|003|031,08|037|006|008|008,246912,"{'id': '846465de-75b4-47b5-84fc-f4bfc1d59d20',...","{'id': 'c12728e7-c4d2-40aa-bbad-e050d3302371',..."
9,08|037|006|002|002,08|037|006|002|022,238789,"{'id': 'b1e91a20-c74f-479e-a510-6d95d6d7eddc',...","{'id': '4011ea88-37ba-4f16-862b-22fa28713e25',..."


In [6]:
from shapely.geometry import shape
# 1. Crea dizionario layerid → (lat, lon)
geo_centroid_dict = {}

for _, row in df_geom.iterrows():
    try:
        geo = row['geojson']  # già dict!
        geom = shape(geo['geometry'])
        centroid = (geom.centroid.y, geom.centroid.x)
        geo_centroid_dict[row['layerid']] = centroid
    except Exception as e:
        print(f"Errore su {row['layerid']}: {e}")
        geo_centroid_dict[row['layerid']] = (None, None)

# 2. Crea nuove colonne facendo il mapping
df['lat1'] = df['ace1'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df['lon1'] = df['ace1'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])
df['lat2']   = df['ace2'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df['lon2']   = df['ace2'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])

In [7]:
import folium
from geopy.distance import geodesic
import branca.colormap as bcm

# Filtra righe valide con coordinate
df_plot = df.dropna(subset=['lat1', 'lon1', 'lat2', 'lon2'])

# Imposta valori min e max dal dataset (top 10)
#min_val = df_plot['total_movements'].min()
min_val = 100000
max_val = df_plot['total_movements'].max()

# Colormap 
linear_colormap = bcm.LinearColormap(
    colors=['#fff5f0', '#fb6a4a', '#67000d'],
    vmin=min_val,
    vmax=max_val,
    caption='Totale spostamenti'
)

# Mappa base
mappa = folium.Map(
    location=[44.5, 11.3],
    zoom_start=8,
    tiles="CartoDB positron"
)

# Aggiunge la legenda
linear_colormap.add_to(mappa)

# Funzione per generare coordinate ad arco
def arc_coordinates(p1, p2, num_points=200, base_height=0.002):
    distance_km = geodesic(p1, p2).km
    height = min(0.05, base_height * distance_km)
    lats = np.linspace(p1[0], p2[0], num_points)
    lons = np.linspace(p1[1], p2[1], num_points)
    curve = height * np.sin(np.linspace(0, np.pi, num_points))
    lats += curve
    return list(zip(lats, lons))

# Aggiunge archi
for _, row in df_plot.iterrows():
    p1 = (row['lat1'], row['lon1'])
    p2 = (row['lat2'], row['lon2'])
    coords = arc_coordinates(p1, p2)
    value = row['total_movements']

    # Colore e peso dalla stessa colormap
    color = linear_colormap(value)
    norm_value = (value - min_val) / (max_val - min_val) if max_val > min_val else 0
    weight = 2 + 8 * norm_value  # Spessore ben visibile

    folium.PolyLine(
        locations=coords,
        weight=weight,
        color=color,
        opacity=0.6,
        tooltip=f"{row['ace1']} → {row['ace2']}: {value:,} spostamenti"
    ).add_to(mappa)

# Visualizza mappa
mappa